In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, pipeline
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch
import os

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

In [3]:
device

'cuda'

In [8]:
model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

In [10]:
%%time
prompt = "Explain AGI ?"
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)
print(pipe(prompt, max_new_tokens=200))


Device set to use cuda:0


[{'generated_text': 'Explain AGI ?\nI am trying to decide between a AI and a human in this situation.\nAI is highly beneficial but I think humans can be too critical.\nHumans can be too critical but AI has many more benefits as well.\nMany people are unhappy or in pain with humans.\nIn the future, humans would love to have AI but I think humans would love AI.\n\nWhat would be the best position for both AI and humans?'}]
CPU times: total: 8.17 s
Wall time: 8.52 s


In [7]:
del model
del tokenizer